In [52]:
import pathlib 
import re 
import random 

# Preparing paths 

In [53]:
listOfFolders = list(pathlib.Path('.').glob("*nm")) # list of folders 
listOfFolders_excl_exec = []       # list of folders not executed yet 


# excluding 15nm and 10nm folders, since they are already executed 
for i in range(len(listOfFolders)): 
    if str(listOfFolders[i])=="10nm" or str(listOfFolders[i])=="15nm": 
        continue
    else: 
        listOfFolders_excl_exec.append(listOfFolders[i])

listOfFolders_excl_exec

[PosixPath('12.5nm'),
 PosixPath('11nm'),
 PosixPath('10.5nm'),
 PosixPath('14nm'),
 PosixPath('12nm'),
 PosixPath('13.5nm'),
 PosixPath('14.5nm'),
 PosixPath('13nm'),
 PosixPath('11.5nm')]

# Functions 

### ar8192.inp template

In [54]:

def render_ar8192_inp(box_size: float): 
    '''render ar8192.inp with differing box sizes
    
    box_size: in nm 
    
    '''

    box_size_å = box_size*10  # Ångström 
    file = f'''#
# Lipid double layer with water over and below
#

# Every atom from diferent molecules will be far from each other at
# least 2.0 Angstrons at the solution.

tolerance 2.0

seed {random.randint(1, 80000)}
nloop0 100

# Coordinate file types will be in pdb format (keyword not required for
# pdb file format, but required for tinker, xyz or moldy).

filetype pdb

# The output pdb file

output {box_size}nm.pdb


structure Ar.pdb
  number 8192
  inside box 0. 0. 0. {box_size_å} {box_size_å} {box_size_å}
end structure
'''
    return file 

### execute_packmol_locally.sh template 

In [55]:
def render_execute_packmol_locally_sh(): 
    file = "#!/bin/bash\n"

    for folder in listOfFolders_excl_exec: 
        file+= f"cd {str(folder)}\n"
        file+= "./packmol < ar8192.inp\n"
        file+= "cd ..\n"  
    return file 

### run.sh template

In [56]:
def render_run_sh(box_size: float): 
    
    # formatting 
    try: # integer 
        box_size= int(str(box_size))
    except ValueError:  # float 
        box_size= float(str(box_size))

        

    file_1part = '''#!/bin/bash
#SBATCH --job-name=slab1
#SBATCH --account=project_2008684
#SBATCH --time=01:00:00
#SBATCH --partition=small
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8

set -o errexit

module load gromacs-env'''


    file_2part = f'''


x={box_size} # nm''' 
    
    file_3part =''' 
export OMP_NUM_THREADS=${SLURM_CPUS_PER_TASK}



# convert to gro and add pbc
gmx_mpi editconf -f ${x}nm.pdb -box $x $x $x -o ${x}nm.gro

# prepare energy minimization
gmx_mpi grompp -f steep.mdp -c ${x}nm.gro -p LJ8192.top -o ${x}nm_steep.tpr

# run energy minimization
srun gmx_mpi mdrun -s -deffnm ${x}nm_steep

# prepare NVT simulation
gmx_mpi grompp -f NVT.mdp -c ${x}nm_steep.gro -p LJ8192.top -o ${x}nm_8192.tpr

# run NVT simulation
srun gmx_mpi mdrun -s -deffnm ${x}nm_8192 -ntomp $OMP_NUM_THREADS


# analyse spinodal
# gmx_mpi density -f ${x}nm_8192.gro -s ${x}nm_8192.tpr -d Y -sl 100 -o density_afterNVT.xvg

'''
    return file_1part+file_2part+file_3part





### execute_runs.sh template

In [57]:
def render_execute_runs_sh(): 
    file = '''#!/bin/bash
# executing each run.sh in each folders 

cd ..
chmod -R u+x Exercise2      # s.t. we can execute our .sh scripts 
cd Exercise2             # move to the folder to start with 

'''
    
    for folder in listOfFolders_excl_exec: 
        file+= f"cd {str(folder)}\n"
        file+= "sbatch run.sh\n"
        file+= "cd ..\n"  

    return file 


In [58]:
listOfFolders_excl_exec

[PosixPath('12.5nm'),
 PosixPath('11nm'),
 PosixPath('10.5nm'),
 PosixPath('14nm'),
 PosixPath('12nm'),
 PosixPath('13.5nm'),
 PosixPath('14.5nm'),
 PosixPath('13nm'),
 PosixPath('11.5nm')]

# Execution 

### Create ar8192.inp for each distances (excl. already executed distances)

In [59]:
listOfFolders_excl_exec

[PosixPath('12.5nm'),
 PosixPath('11nm'),
 PosixPath('10.5nm'),
 PosixPath('14nm'),
 PosixPath('12nm'),
 PosixPath('13.5nm'),
 PosixPath('14.5nm'),
 PosixPath('13nm'),
 PosixPath('11.5nm')]

In [60]:
for nm_folder in listOfFolders_excl_exec: 
    length = float(re.search(r'(\d+\.?\d?)', str(nm_folder)).groups()[0]) # length of the box 
    
    # write to file 
    with open(str(nm_folder/"ar8192.inp"), 'w') as file: 
        file.write(render_ar8192_inp(length))

### Create .sh files for each distances (excl. already executed distances)

In [61]:
for nm_folder in listOfFolders_excl_exec: 
    length = float(re.search(r'(\d+\.?\d?)', str(nm_folder)).groups()[0]) # length of the box 
    
    # write to file 
    with open(str(nm_folder/"run.sh"), 'w') as file: 
        file.write(render_run_sh(length))

### Create main .sh file for executing all run.sh under each distances 

In [62]:
with open("execute_runs.sh", "w") as file: 
    file.write(render_execute_runs_sh())

### Create .sh file for running packmol to generate .pdb files 

In [63]:
with open("execute_packmol_locally.sh", "w") as file: 
    file.write(render_execute_packmol_locally_sh())
